In [1]:
from pynq.overlays.base import BaseOverlay
import time
import multiprocessing
import socket
import os

base = BaseOverlay("base.bit")

In [3]:
%%microblaze base.PMODA

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODA
void write_gpioA(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODA
unsigned int read_gpioA(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMOD
void reset_gpioA(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpioA(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [4]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"

//Function to turn on/off a selected pin of PMODB
void write_gpioB(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
}

//Function to read the value of a selected pin of PMODB
unsigned int read_gpioB(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Add a C++ function to reset all the GPIO pins on the chosen PMODB
void reset_gpioB(){
    for (unsigned int j = 0; j <= 7; j = j + 1) { 
        write_gpioB(j, 0);
        //write_gpio(pin,val)
        //we want the value reset to zero
        //12 pins, starts at 0
    }
}

In [5]:
freq = 10
beep_time = int(freq/2)
#print(type(beep_time))

In [6]:
def BeepBoop(PMOD):
    #pseudocode:
    #write gpio high
    #time.sleep(1/(2*freq))
    #write gpio low
    #time.sleep(1/(2*freq))
    #note: We want it to beep for 0.5s, so:       number of beeps = freq/2
    global freq, beep_time
    print('freq,beep_time',type(freq),type(beep_time))
    if PMOD == 'A': #Beep on GPIO A!
        for i in range(beep_time):
            write_gpioA(0,1)
            time.sleep(1/(2*freq))
            write_gpioA(0,0)
            time.sleep(1/(2*freq))
    else: #Beep on GPIO B!
        for i in range(beep_time):
            write_gpioB(0,1)
            time.sleep(1/(2*freq))
            write_gpioB(0,0)
            time.sleep(1/(2*freq))
    print('Beeped PMOD',PMOD)

In [7]:
port1 = 12345
#server1_name = '192.168.2.99' #pynq board
#server1_name = '192.168.2.1' #laptop
server1_name = '127.0.0.1' #localhost

def Server1():
    global port1
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(('0.0.0.0', port1))
    sock.listen()
    print('Server1 is waiting for connection')
    conn, addr = sock.accept()
    print('Connected to Server1')
    pmodA = 'A'
    with conn:
        while True:
            data = conn.recv(1024)
            print('Prior to decoding, received data is: ',data)
            print('data is: ', type(data))
            data = data.decode()
            print('After decoding, received data is:    ',data)
            print('Data is: ',type(data))
            if data == '1':
                BeepBoop(pmodA)
                print('Server1 is now buzzing!')
            elif data == '2':
                break
    print('User is disconnected from Server1')

def Client1():
    global port1, server1_name
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_name = '192.168.2.1' #Local machine
    print('Client1 is waiting to press button0 to connect to server.')
    while True:
        if base.buttons[0].read() != 0:
            print('Client1 has pressed button0.')
            sock.connect((server1_name, port1)) #Connect to server
            print('Client1 has connected to the server.')
            break
        time.sleep(.25)
    while True:
        if base.buttons[1].read() != 0:
            user_input = '1' #string!
            sock.sendall(bytes(user_input, 'utf-8'))
            print('Client1 sent a buzzer request!')
            time.sleep(.5)
        elif base.buttons[2].read() != 0:
            user_input = '2' #string!
            sock.sendall(bytes(user_input, 'utf-8'))
            break
    sock.close()
    print('Client1 closed the socket!')

    
procs = []
#Server1 will run on CPU0
p1 = multiprocessing.Process(target=Server1) # the first arg defines which CPU to run the 'target' on
os.system("taskset -p -c {} {}".format(0, p1.pid)) # taskset is an os command to pin the process to a specific CPU
p1.start() # start the process
procs.append(p1)

#Client1 will run on CPU1
p2 = multiprocessing.Process(target=Client1) # the first arg defines which CPU to run the 'target' on
os.system("taskset -p -c {} {}".format(1, p2.pid)) # taskset is an os command to pin the process to a specific CPU
p2.start() # start the process
procs.append(p2)

for p in procs:
    pName = p.name # get process name
    p.join() # wait for the process to finish
    print('{} is finished'.format(pName))

Server1 is waiting for connection

Client1 is waiting to press button0 to connect to server.Client1 has pressed button0.
Client1 has connected to the server.Connected to Server1

Prior to decoding, received data is: Client1 sent a buzzer request! 
b'1'
data is:  <class 'bytes'>
After decoding, received data is:     1
Data is:  <class 'str'>
freq,beep_time <class 'int'> <class 'int'>
Beeped PMOD A
Server1 is now buzzing!
Prior to decoding, received data is: Client1 sent a buzzer request! 
b'1'
data is:  <class 'bytes'>
After decoding, received data is:     1
Data is:  <class 'str'>
freq,beep_time <class 'int'> <class 'int'>
Beeped PMOD A
Server1 is now buzzing!
Prior to decoding, received data is: Client1 sent a buzzer request!
 b'1'
data is:  <class 'bytes'>
After decoding, received data is:     1
Data is:  <class 'str'>
freq,beep_time <class 'int'> <class 'int'>
Beeped PMOD A
Server1 is now buzzing!
Prior to decoding, received data is: Client1 sent a buzzer request! 
b'1'
data is:  <c